<a href="https://colab.research.google.com/github/mmarkaki/learning/blob/master/Stress_recognition_in_Greek_speech_using_Wav2Vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotion Recognition in Greek Speech Using Wav2Vec 2.0

**Wav2Vec 2.0** is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2`s ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

During fine-tuning week hosted by HuggingFace, more than 300 people participated in tuning XLSR-Wav2Vec2's pretrained on low-resources ASR dataset for more than 50 languages. This model is fine-tuned using [Connectionist Temporal Classification](https://distill.pub/2017/ctc/) (CTC), an algorithm used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. Follow this [notebook](https://colab.research.google.com/github/patrickvonplaten/notebooks/blob/master/Fine_Tune_XLSR_Wav2Vec2_on_Turkish_ASR_with_%F0%9F%A4%97_Transformers.ipynb#scrollTo=Gx9OdDYrCtQ1) for more information about XLSR-Wav2Vec2 fine-tuning.

This model was shown significant results in many low-resources languages. You can see the [competition board](https://paperswithcode.com/dataset/common-voice) or even testing the models from the [HuggingFace hub](https://huggingface.co/models?filter=xlsr-fine-tuning-week).


In this notebook, we will go through how to use this model to recognize the emotional aspects of speech in a language (or even as a general view using for every classification problem). Before going any further, we need to install some handy packages and define some enviroment values.

In [ ]:
!ps -elf | grep python

4 Z root          74       7  0  80   0 -     0 -      11:04 ?        00:00:06 [python3] <defunct>
4 S root          75       7  0  80   0 - 13661 do_epo 11:04 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
4 S root          88       7  0  80   0 - 209819 do_epo 11:04 ?       00:00:03 /usr/bin/python3 /usr/local/bin/jupyter-notebook --debug --transport="ipc" --ip=172.28.0.12 --NotebookApp.token= --port=9000 --FileContentsManager.root_dir=/ --MappingKernelManager.root_dir=/content
4 S root        5080      88 29  80   0 - 334443 do_sel 11:25 ?       00:00:02 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-8c81258c-3481-4f1a-a26f-4365d25d258f.json
1 S root        5122       1  0  80   0 - 79171 futex_ 11:25 ?        00:00:00 /usr/bin/python3 /usr/local/lib/python3.10/dist-packages/debugpy/adapter --for-server 58181 --host 127.0.0.1 --port 42273 --server-access-token ee2770975ef49172fce431e9a99616d8950036fc19a51b53950d5148bf7aac42
0 S root  

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul 11 11:25:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture

!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install jiwer
!pip install torchaudio
!pip install librosa

# Monitor the training process
# !pip install wandb

In [ ]:
#from psutil import virtual_memory
#ram_gb = virtual_memory().total / 1e9
#print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#if ram_gb < 20:
 # print('Not using a high-RAM runtime')
#else:
#  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
del(virtual_memory)

In [ ]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=/content/cache
%env HF_DATASETS_CACHE=/content/cache
%env CUDA_LAUNCH_BLOCKING=1

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=/content/cache
env: HF_DATASETS_CACHE=/content/cache
env: CUDA_LAUNCH_BLOCKING=1


In [ ]:
# # Uncomment this part if you want to setup your wandb project

# %env WANDB_WATCH=all
# %env WANDB_LOG_MODEL=1
# %env WANDB_PROJECT=YOUR_PROJECT_NAME
# !wandb login YOUR_API_KEY --relogin

## Prepare Data

For this particular example, we use [Acted Emotional Speech Dynamic Database – AESDD](http://m3c.web.auth.gr/research/aesdd-speech-emotion-recognition/) provided by Multidisciplinary Media & Mediated Communication Research Group ([M3C](http://m3c.web.auth.gr/)).

The Acted Emotional Speech Dynamic Database (AESDD) is a publically available speech emotion recognition dataset that contains utterances of acted emotional speech in the Greek language for five different emotions `sadness`, `disgust`, `happiness`, `anger`, and `fear`.

The dataset consists of directories of emotions; each folder includes specific emotions. We need to loop over directories and save the paths related to each class based on the directory name.


```bash
.
├── Tools\ and\ Documentation
│   ├── ESTrainer.mlapp
│   ├── Speech\ Emotion\ Recognition\ Adapted\ to\ Multimodal\ Semantic\ Repositories_documentation.pdf
│   ├── Speech\ Emotion\ Recognition\ for\ Performance\ Interaction.pdf
│   └── readme.txt
├── anger
│   ├── a01\ (1).wav
│   ├── a01\ (2).wav
│   ├── ...
├── disgust
│   ├── d01\ (1).wav
│   ├── d01\ (2).wav
│   ├── ...
├── fear
│   ├── f01\ (1).wav
│   ├── f01\ (2).wav
│   ├── ...
├── happiness
│   ├── h01\ (1).wav
│   ├── h01\ (2).wav
│   ├── ...
└── sadness
    ├── s01\ (1).wav
    ├── s01\ (2).wav
    ├── ...

6 directories, 609 files
```

Since the dataset is hosted on [Mega](https://mega.nz/#F!0ShVXY7C!-73kVoK05OjTPEA95UUvMw) and unable to access it directly, we upload the dataset on the G-drive to make this tutorial accessible.

We change the database to the one provided by Giannakakis, which contains stressfull and non-stressfull 2-minutes edited conversations; we have saved these files to google drive : MyDrive/Markaki Stress Speech



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Download the dataset from
#!mkdir -p /content/data
!mkdir -p /content/data1
#!gdown https://drive.google.com/uc?id=1_IAWexEWpH-ly_JaA5EGfZDp-_3flkN1
#!unzip -q aesdd.zip -d /content/data/
#!mv "/content/data/Acted Emotional Speech Dynamic Database/" /content/data/aesdd/
!cp -r "/content/drive/MyDrive/Markaki Stress Speech/class_recs/" /content/data1/aesdd/

In [ ]:
#!cp -r "/content/drive/MyDrive/Markaki Stress Speech/class_recs/Stressed/P0019" /content/data1/aesdd/Stressed/P0019
#I could unmount drive then


In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="max_split_size_mb:512"

import sys

#import audiofile
import glob


I insert here code from Thanos in order to read stressed and neutral wav files

In [ ]:
import librosa
import IPython.display as ipd
# ! pip install samplerate
# ! pip install resampy

data = []

ids= np.arange(1,60)
ids = np.delete(ids,16)
tasks = [2,6]
boys = np.array([2, 6, 7, 8, 9, 13, 16, 19, 24, 32, 33, 34, 37, 38, 39, 40, 41, 42, 48, 49, 50, 51, 52, 53, 55])
girls = np.array([1, 3, 4, 5, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 35, 36, 43, 44, 45, 46, 47, 54, 56, 57, 58, 59])
zero=[]
df0 = pd.DataFrame()
subject = []
tasks=[]
sex=[]
stressed=[]
pathid = []
for i in ids:
    path = "/content/data1/aesdd/Neutral/P00"+str(i)+"/"
    files = os.listdir(path)
    for filename in glob.glob(os.path.join(path, '*.wav')):
        sbj = path.split('/')[5].split('\\')[0].replace('P00','')
        sbj= int(sbj)
        subject.append(sbj)
        pathid.append(filename)
        if sbj in girls:
           sex.append(1)
        else:
             sex.append(0)
        task = int(filename.split('/')[6].split('\\')[0].split('_')[0].replace('tsk0',''))
        tasks.append(task)
        signal, sampling_rate = torchaudio.load(filename)
        zero.append(signal[0:15*sampling_rate-1])
        stressed.append(0)
#for i in zero:
    # The smile.process_signal(
    #     i,
    #     sampling_rate
    # ) is where we do the feature extraction with opensmile. The output is a pandas dataframe
    #df1 = pd.concat([df1.loc[:],smile.process_signal(i,sampling_rate)])
  #  print(df1.loc[:])
  #  df1 = pd.concat([df1.loc[:],sampling_rate])

df0.insert(0,'Subject', subject )
df0.insert(1,'Sex',sex)
df0.insert(2,'Task', tasks )
df0.insert(3,'Stressed', stressed)
df0.insert(4,'Path', pathid)
df0 = df0.reset_index()
#df1 = df1.drop(columns=['start','end'])
#save the dataframe as a csv file
df0.to_csv("Neutral1.csv")
df0.head()

,index,Subject,Sex,Task,Stressed,Path
0,0,1,1,2,0,/content/data1/aesdd/Neutral/P001/tsk02_audio.wav
1,1,1,1,6,0,/content/data1/aesdd/Neutral/P001/tsk06_audio.wav
2,2,2,0,2,0,/content/data1/aesdd/Neutral/P002/tsk02_audio.wav
3,3,2,0,6,0,/content/data1/aesdd/Neutral/P002/tsk06_audio.wav
4,4,3,1,2,0,/content/data1/aesdd/Neutral/P003/tsk02_audio.wav


In [ ]:
data = []

ids= np.arange(1,60)
ids = np.delete(ids,16)
tasks = [2,6]
boys = np.array([2, 6, 7, 8, 9, 13, 16, 19, 24, 32, 33, 34, 37, 38, 39, 40, 41, 42, 48, 49, 50, 51, 52, 53, 55])
girls = np.array([1, 3, 4, 5, 10, 11, 12, 14, 15, 17, 18, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 35, 36, 43, 44, 45, 46, 47, 54, 56, 57, 58, 59])
zero=[]
df1 = pd.DataFrame()
subject = []
pathid = []
tasks=[]
sex=[]
stressed=[]
for i in ids:
    path = "/content/data1/aesdd/Stressed/P00"+str(i)+"/"
    files = os.listdir(path)
    for filename in glob.glob(os.path.join(path, '*.wav')):
        sbj = path.split('/')[5].split('\\')[0].replace('P00','')
        sbj= int(sbj)
        subject.append(sbj)
        pathid.append(filename)
        if sbj in girls:
           sex.append(1)
        else:
             sex.append(0)
        task = int(filename.split('/')[6].split('\\')[0].split('_')[0].replace('tsk0',''))
        tasks.append(task)
        signal, sampling_rate = torchaudio.load(filename)
        zero.append(signal[0:15*sampling_rate-1])
        stressed.append(1)
#for i in zero:
    # The smile.process_signal(
    #     i,
    #     sampling_rate
    # ) is where we do the feature extraction with opensmile. The output is a pandas dataframe
    #df1 = pd.concat([df1.loc[:],smile.process_signal(i,sampling_rate)])
  #  print(df1.loc[:])
  #  df1 = pd.concat([df1.loc[:],sampling_rate])

df1.insert(0,'Subject', subject )
df1.insert(1,'Sex',sex)
df1.insert(2,'Task', tasks )
df1.insert(3,'Stressed', stressed)
df1.insert(4,'Path', pathid)
#path in tqdm(Path("/content/data1/aesdd").glob("**/*.wav")):
df1 = df1.reset_index()
#df1 = df1.drop(columns=['start','end'])
#save the dataframe as a csv file
df1.to_csv("Stressed1.csv")
df1.head()

,index,Subject,Sex,Task,Stressed,Path
0,0,1,1,7,1,/content/data1/aesdd/Stressed/P001/tsk07_audio...
1,1,1,1,8,1,/content/data1/aesdd/Stressed/P001/tsk08_audio...
2,2,1,1,3,1,/content/data1/aesdd/Stressed/P001/tsk03_audio...
3,3,2,0,7,1,/content/data1/aesdd/Stressed/P002/tsk07_audio...
4,4,2,0,8,1,/content/data1/aesdd/Stressed/P002/tsk08_audio...


In [ ]:
#data = []

#for path in tqdm(Path("/content/data1/aesdd").glob("**/*.wav")):
    #name = str(path).split('/')[-1].split('.')[0]
    #label = str(path).split('/')[-2]

    #try:
        # There are some broken files
        #s = torchaudio.load(path)
        #data.append({
        #    "name": name,
       #     "path": path,
      #      "emotion": label
     #   })
    #except Exception as e:
        # print(str(path), e)
   #     pass

    # break

605it [00:01, 563.27it/s]


In [ ]:
# Filter broken and non-existed paths

#print(f"Step 0: {len(df)}")

#df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
#df = df.dropna(subset=["path"])
#df = df.drop("status", 1)
#print(f"Step 1: {len(df)}")

#df = df.sample(frac=1)
#df = df.reset_index(drop=True)
#df.head()

Let's explore how many labels (emotions) are in the dataset with what distribution.

In [ ]:
#print("Labels: ", df["emotion"].unique())
#print()
#df.groupby("emotion").count()[["path"]]
df0.groupby("Task").count()
df1.groupby("Task").count()
result = [df0, df1]

df = pd.concat(result)
df.groupby("Task").count()
display(df)

df = df.reset_index()
#save the dataframe as a csv file
df.to_csv("NeutralStressed.csv")
#df.head()


,index,Subject,Sex,Task,Stressed,Path
0,0,1,1,2,0,/content/data1/aesdd/Neutral/P001/tsk02_audio.wav
1,1,1,1,6,0,/content/data1/aesdd/Neutral/P001/tsk06_audio.wav
2,2,2,0,2,0,/content/data1/aesdd/Neutral/P002/tsk02_audio.wav
3,3,2,0,6,0,/content/data1/aesdd/Neutral/P002/tsk06_audio.wav
4,4,3,1,2,0,/content/data1/aesdd/Neutral/P003/tsk02_audio.wav
...,...,...,...,...,...,...
169,169,58,1,8,1,/content/data1/aesdd/Stressed/P0058/tsk08_audi...
170,170,58,1,3,1,/content/data1/aesdd/Stressed/P0058/tsk03_audi...
171,171,59,1,7,1,/content/data1/aesdd/Stressed/P0059/tsk07_audi...
172,172,59,1,8,1,/content/data1/aesdd/Stressed/P0059/tsk08_audi...


Let's display some random sample of the dataset and run it a couple of times to get a feeling for the audio and the emotional label.

In [ ]:
idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
print(sample)
path = sample["Path"]
label = sample["Stressed"]

print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)
print(sr)
speech = np.asarray(speech[0].numpy().squeeze())
print(len(speech))#I want this to be at most 1 minute, or 60*sr samples
#speech = librosa.resample(speech,orig_sr=sr,target_sr=16000) #, sr, 16_000)
ipd.Audio(data=np.asarray(speech[0:4*sr-1]), autoplay=True, rate=16000)

level_0                                                    57
index                                                      57
Subject                                                    30
Sex                                                         1
Task                                                        6
Stressed                                                    0
Path        /content/data1/aesdd/Neutral/P0030/tsk06_audio...
Name: 57, dtype: object
ID Location: 57
      Label: 0

16000
1920755


For training purposes, we need to split data into train test sets; in this specific example, we break with a `20%` or `30% - 40%` rate for the test set.

In [ ]:
save_path = "/content/data1"

#train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["Stressed"])
train_df, test_df = train_test_split(df, test_size=0.3, random_state=101, stratify=df["Stressed"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(174, 7)
(116, 7)


We also wish to perform classification  according to https://github.com/audeering/w2v2-how-to/blob/main/notebook.ipynb: As classifier, we use a Support Vector Classification from scikit-learn. Since Emo-DB does not define an official train and test set, we apply leave-one-speaker-out, i.e. we predict each speaker individually after training on the other speakers. For this purpose, we define the following utility function.

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


# create classifier and grouping object
clf = make_pipeline(
    StandardScaler(),
    SVC(gamma='auto'),
)
logo = LeaveOneGroupOut()

def experiment(
    features,
    targets,
    groups,
):
    truths = []
    preds = []

    # leave-one-speaker loop
    pbar = audeer.progress_bar(
        total=len(groups.unique()),
        desc='Run experiment',
    )
    for train_index, test_index in logo.split(
        features,
        targets,
        groups=groups,
    ):
        train_x = features.iloc[train_index]
        train_y = targets[train_index]
        clf.fit(train_x, train_y)

        truth_x = features.iloc[test_index]
        truth_y = targets[test_index]
        predict_y = clf.predict(truth_x)

        truths.append(truth_y)
        preds.append(predict_y)

        pbar.update()

    # combine speaker folds
    truth = pd.concat(truths)
    truth.name = 'truth'
    pred = pd.Series(
        np.concatenate(preds),
        index=truth.index,
        name='prediction',
    )

    return truth, pred



After passing the features to the utility function, we get ground truth and predictions for every sentence in the database.

## Prepare Data for Training

In [ ]:
# Loading the created dataset using datasets; also saved them in Mydrive
from datasets import load_dataset, load_metric

data_files = {
    "train": "/content/data1/train.csv",
    "validation": "/content/data1/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-94c9f63ee11d69a0/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['level_0', 'index', 'Subject', 'Sex', 'Task', 'Stressed', 'Path'],
    num_rows: 174
})
Dataset({
    features: ['level_0', 'index', 'Subject', 'Sex', 'Task', 'Stressed', 'Path'],
    num_rows: 116
})


In [ ]:
# We need to specify the input and output column
input_column = "Path"
output_column = "Stressed"

In [ ]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 2 classes: [0, 1]


In order to preprocess the audio into our classification model, we need to set up the relevant Wav2Vec2 assets regarding our language in this case `lighteternal/wav2vec2-large-xlsr-53-greek` fine-tuned by [Dimitris Papadopoulos](https://huggingface.co/lighteternal/wav2vec2-large-xlsr-53-greek). To handle the context representations in any audio length we use a merge strategy plan (pooling mode) to concatenate that 3D representations into 2D representations.

There are three merge strategies `mean`, `sum`, and `max`. In this example, we achieved better results on the mean approach. In the following, we need to initiate the config and the feature extractor from the Dimitris model.

In [ ]:
from transformers import AutoConfig, Wav2Vec2Processor

In [ ]:
model_name_or_path = "lighteternal/wav2vec2-large-xlsr-53-greek"
pooling_mode = "mean"

In [ ]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)
#if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The target sampling rate: 16000


# Preprocess Data

So far, we downloaded, loaded, and split the SER dataset into train and test sets. The instantiated our strategy configuration for using context representations in our classification problem SER. Now, we need to extract features from the audio path in context representation tensors and feed them into our classification model to determine the emotion in the speech.

Since the audio file is saved in the `.wav` format, it is easy to use **[Librosa](https://librosa.org/doc/latest/index.html)** or others, but we suppose that the format may be in the `.mp3` format in case of generality. We found that the **[Torchaudio](https://pytorch.org/audio/stable/index.html)** library works best for reading in `.mp3` data.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a **map(...)** function accordingly. Also, we need to handle the string labels into integers for our specific classification task in this case, the **single-label classification** you may want to use for your **regression** or even **multi-label classification**.

In [ ]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array[0:4*sampling_rate-1]).squeeze().numpy()
    return speech

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [ ]:
#it was batch_size=100; reduced training samples to 70% from 80%;reduced sampling rate to 8kHz
pip install koila
#https://github.com/rentruewang/koila
#Define the input, label, and model:
# A batch of MNIST image
#input = torch.randn(8, 28, 28)
# A batch of labels
#label = torch.randn(0, 10, [8])
#class NeuralNetwork(Module):
#Define the loss function, calculate output and losses.
#loss_fn = CrossEntropyLoss()#this is what I have
# Calculate losses
#out = nn(t)
#loss = loss_fn(out, label)
# Backward pass
#nn.zero_grad()
#loss.backward()
# Wrap the input tensor and label tensor.
# If a batch argument is provided, that dimension of the tensor would be treated as the batch.
# In this case, the first dimension (dim=0) is used as batch's dimension.
(input, label) = lazy(input, label, batch=0)



In [ ]:

train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=1,#power of 2, 16 instead?it was 4?100?
    batched=True,
    num_proc=1#4
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=1,#power of 2
    batched=True,
    num_proc=1#4
)

Map:   0%|          | 0/174 [00:00<?, ? examples/s]

Map:   0%|          | 0/116 [00:00<?, ? examples/s]

In [ ]:
idx = 0
#print(f"Training input_values: {train_dataset[idx]['input_values']}")
#print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['Stressed']}")

Training labels: 1 - 1


Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

## Model

Before diving into the training part, we need to build our classification model based on the merge strategy.

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
torch.cuda.empty_cache()
#drive.flush_and_unmount()
from transformers.file_utils import ModelOutput



@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [ ]:
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
#other losses too?
#! pip install audmetric
import audmetric
#audmetric.unweighted_average_recall(truth_w2v2, pred_w2v2)

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
               # total_loss += loss.item()
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()#audmetric.unweighted_average_recall()
                #audmetric.unweighted_average_recall(truth_w2v2, pred_w2v2)
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
   # total_loss += loss.item()

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

Check w2v2-how-to for proper metric

In [ ]:
is_regression = False

In [ ]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:#audmetric.unweighted_average_recall(truth_w2v2, pred_w2v2)
      # return {"UAR": ()}
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained XLSR-Wav2Vec2 checkpoint into our classification model with a pooling strategy.

In [ ]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at lighteternal/wav2vec2-large-xlsr-53-greek and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [ ]:
#from google.colab import drive
#drive.flush_and_unmount()

#checkpoint = {
    #    'epoch': epoch,
   #     'model_state_dict': model.state_dict(),
  #      'optimizer_state_dict': optimizer.state_dict(),
 #       # Add any other information you want to save
#}
#torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
#! pip install transformers[torch]
#! pip install accelerate -U
from transformers import TrainingArguments
import accelerate

training_args = TrainingArguments(
    output_dir="/content/wav2vec2-xlsr-greek-speech-stress-recognition",
    #output_dir="/content/drive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition",
    per_device_train_batch_size=1,#2?4
    per_device_eval_batch_size=1,#2?4
    gradient_accumulation_steps=1,#2
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,#better than fp32
    save_steps=2,
    eval_steps=2,#10 in all steps
    logging_steps=2,
    learning_rate=1e-4,
    save_total_limit=2,
)

For future use we can create our training script, we do it in a simple way. You can add more on you own.

In [ ]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)
# ! pip install apex
if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_cuda_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_cuda_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


In [ ]:
#from torchsummary import summary
#summary(model)

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

running on the GPU


Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,

)

### Training

Training will take between 10 and 60 minutes depending on the GPU allocated to this notebook.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [ ]:
# --W 256 --H 256
#torch.cuda.memory_summary(device=None, abbreviated=False)
print(torch.cuda.memory_summary(device=None, abbreviated=False))
import gc
#del speech #variables#all useless variables
#del signal
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 13           |        cudaMalloc retries: 47        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  13534 MiB |  13608 MiB |   1369 GiB |   1355 GiB |
|       from large pool |  13530 MiB |  13605 MiB |   1368 GiB |   1355 GiB |
|       from small pool |      3 MiB |      7 MiB |      0 GiB |      0 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  13534 MiB |  13608 MiB |   1369 GiB |   1355 GiB |
|       from large pool |  13530 MiB |  13605 MiB |   1368 GiB |

In [ ]:
#import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="max_split_size_mb:512"
trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1537 in train                    │
│                                                                                                  │
│   1534 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1535 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1536 │   │   )                                                                                 │
│ ❱ 1537 │   │   return inner_training_loop(                                                       │
│   1538 │   │   │   args=args,                                                                    │
│   1539 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1540 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:1802 in _inner_training_loop     │
│                                                                                                  │
│   1799 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1800 │   │   │   │                                                                             │
│   1801 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1802 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1803 │   │   │   │                                                                             │
│   1804 │   │   │   │   if (                                                                      │
│   1805 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│ in training_step:47                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/trainer.py:2672 in compute_loss             │
│                                                                                                  │
│   2669 │   │   │   labels = inputs.pop("labels")                                                 │
│   2670 │   │   else:                                                                             │
│   2671 │   │   │   labels = None                                                                 │
│ ❱ 2672 │   │   outputs = model(**inputs)                                                         │
│   2673 │   │   # Save past state if it exists                                                    │
│   2674 │   │   # TODO: this needs to be fixed and made cleaner later.                            │
│   2675 │   │   if self.args.past_index >= 0:                                                     │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 12           |        cudaMalloc retries: 46        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  13264 MiB |  13608 MiB |   1368 GiB |   1355 GiB |
|       from large pool |  13260 MiB |  13605 MiB |   1368 GiB |   1355 GiB |
|       from small pool |      3 MiB |      7 MiB |      0 GiB |      0 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  13264 MiB |  13608 MiB |   1368 GiB |   1355 GiB |
|       from large pool |  13260 MiB |  13605 MiB |   1368 GiB |

The training loss goes down and we can see that the Acurracy on the test set also improves nicely. Because this notebook is just for demonstration purposes, we can stop here.

The resulting model of this notebook has been saved to [m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition](https://huggingface.co/m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition)

As a final check, let's load the model and verify that it indeed has learned to recognize the emotion in the speech.

Let's first load the pretrained checkpoint.

## Evaluation

In [ ]:
import librosa
from sklearn.metrics import classification_report

In [ ]:
test_dataset = load_dataset("csv", data_files={"test": "/content/data/test.csv"}, delimiter="\t")["test"]
test_dataset

NameError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [ ]:
model_name_or_path = "m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), orig_sr=sampling_rate, target_sr=processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [ ]:
result = test_dataset.map(predict, batched=True, batch_size=8)

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

['anger', 'disgust', 'fear', 'happiness', 'sadness']

In [ ]:
y_true = [config.label2id[name] for name in result["emotion"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

[1, 4, 4, 3, 4]
[1, 4, 4, 3, 4]


In [ ]:
print(classification_report(y_true, y_pred, target_names=label_names))

              precision    recall  f1-score   support

       anger       1.00      0.98      0.99        48
     disgust       0.98      1.00      0.99        49
        fear       0.98      0.98      0.98        48
   happiness       0.98      0.96      0.97        48
     sadness       0.98      1.00      0.99        49

    accuracy                           0.98       242
   macro avg       0.98      0.98      0.98       242
weighted avg       0.98      0.98      0.98       242



# Prediction

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "m3hrdadfi/wav2vec2-xlsr-greek-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


STYLES = """
<style>
div.display_data {
    margin: 0 auto;
    max-width: 500px;
}
table.xxx {
    margin: 50px !important;
    float: right !important;
    clear: both !important;
}
table.xxx td {
    min-width: 300px !important;
    text-align: center !important;
}
</style>
""".strip()

def prediction(df_row):
    path, emotion = df_row["path"], df_row["emotion"]
    df = pd.DataFrame([{"Emotion": emotion, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    ipd.display(ipd.HTML(STYLES + df.to_html(**setup) + "<br />"))
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), orig_sr=sr, target_sr=sampling_rate)
    ipd.display(ipd.Audio(data=np.asarray(speech), autoplay=True, rate=sampling_rate))

    outputs = predict(path, sampling_rate)
    r = pd.DataFrame(outputs)
    ipd.display(ipd.HTML(STYLES + r.to_html(**setup) + "<br />"))

In [ ]:
test = pd.read_csv("/content/data/test.csv", sep="\t")
test.head()

,name,path,emotion
0,d18 (3),/content/data/aesdd/Acted Emotional Speech Dyn...,disgust
1,s19 (5),/content/data/aesdd/Acted Emotional Speech Dyn...,sadness
2,s20 (4),/content/data/aesdd/Acted Emotional Speech Dyn...,sadness
3,h17 (6),/content/data/aesdd/happiness/h17 (6).wav,happiness
4,s12 (1),/content/data/aesdd/Acted Emotional Speech Dyn...,sadness


In [ ]:
prediction(test.iloc[0])

,Emotion,Sentence
0,disgust,


,Emotion,Score
0,anger,0.0%
1,disgust,99.2%
2,fear,0.1%
3,happiness,0.1%
4,sadness,0.7%


In [ ]:
prediction(test.iloc[1])

,Emotion,Sentence
0,sadness,


,Emotion,Score
0,anger,0.3%
1,disgust,0.3%
2,fear,0.1%
3,happiness,0.0%
4,sadness,99.4%


In [ ]:
prediction(test.iloc[2])

,Emotion,Sentence
0,sadness,


,Emotion,Score
0,anger,0.3%
1,disgust,0.1%
2,fear,0.6%
3,happiness,0.0%
4,sadness,99.0%
